# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with Import the dataset with:
`mongoimport --db uk_food --collection establishments --file Resources/establishments.json --jsonArray`


In [1]:
!pip install pymongo
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
client = MongoClient()

In [3]:
# confirm that our new database was created
print(client.list_database_names())  # Esto mostrará las bases de datos disponibles
db = client.uk_food  # Conecta con la base de datos
print(db.list_collection_names())  # Verifica que 'establishments' aparece



['admin', 'config', 'local', 'uk_food']
['establishments']


In [9]:
# Assign the uk_food database to a variable name
db = client['uk_food']


In [11]:
# review the collections in our new database
print(db.list_collection_names())



['establishments']


In [13]:
# review the collections in our new database
from pprint import pprint

pprint(db.establishments.find_one())



{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677b1ecc484c06fc3faa5af8'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [19]:
# review a document in the establishments collection
establishments = db['establishments']
# Check how many documents are in the collection
print(establishments.count_documents({}))



39779


In [23]:
# assign the collection to a variable
establishments = db['establishments']



## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [25]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "",
    "RatingValue": None,
    "AddressLine1": "Greenwich",
    "AddressLine2": "",
    "PostCode": "",
    "longitude": None,
    "latitude": None,
    "scores": None
}


In [27]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)



InsertOneResult(ObjectId('677b217e731046826a3d254a'), acknowledged=True)

In [29]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))



{'AddressLine1': 'Greenwich',
 'AddressLine2': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': '',
 'PostCode': '',
 'RatingValue': None,
 '_id': ObjectId('677b217e731046826a3d254a'),
 'latitude': None,
 'longitude': None,
 'scores': None}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [31]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(business_type)



{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [33]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [35]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))



{'AddressLine1': 'Greenwich',
 'AddressLine2': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': '',
 'BusinessTypeID': 1,
 'PostCode': '',
 'RatingValue': None,
 '_id': ObjectId('677b217e731046826a3d254a'),
 'latitude': None,
 'longitude': None,
 'scores': None}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [37]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")



Number of documents with LocalAuthorityName as 'Dover': 994


In [39]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})



DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [41]:
# Check if any remaining documents include Dover
remaining_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Remaining documents with LocalAuthorityName as 'Dover': {remaining_dover}")



Remaining documents with LocalAuthorityName as 'Dover': 0


In [43]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())



{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677b1ecc484c06fc3faa5ddd'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289353',
       

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [45]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {"$set": {"longitude": {"$toDouble": "$longitude"}}},
        {"$set": {"latitude": {"$toDouble": "$latitude"}}}
    ]
)



UpdateResult({'n': 38786, 'nModified': 38785, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [50]:
# Find all documents where RatingValue is a string
documents = establishments.find({"RatingValue": {"$type": "string"}})

# Iterate over the documents and convert RatingValue to an integer
for doc in documents:
    if doc["RatingValue"].isdigit():  # Check if it's a valid number
        establishments.update_one(
            {"_id": doc["_id"]},
            {"$set": {"RatingValue": int(doc["RatingValue"])}}
        )

# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},  # Find documents where RatingValue is stored as a string
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  # Convert RatingValue to integer
)


In [52]:
# Check that the coordinates and rating value are now numbers
print(establishments.find_one({"RatingValue": {"$type": "int"}}))

# Check if coordinates (latitude and longitude) are stored as decimal numbers
print(establishments.find_one({
    "geocode.latitude": {"$type": "double"}, 
    "geocode.longitude": {"$type": "double"}
}))


{'_id': ObjectId('677b1ecc484c06fc3faa5ddd'), 'FHRSID': 289353, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000002468', 'BusinessName': 'St Marys COE (aided) Primary School', 'BusinessType': 'School/college/university', 'BusinessTypeID': 7845, 'AddressLine1': 'St Marys Church Of England Primary School', 'AddressLine2': 'Warren Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6QH', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2016-09-13T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.194762', 'latitude': '51.085797'}, 'RightToReply': '', 'Distance': 4591.871474174837, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractD